# Projet Diversité et inclusion en entreprise #

**Problématique : Comment les différents aspects sociaux et professionnels (rémunération, formation, conditions de travail, handicap, absentéisme, etc.) influencent-ils la structure et l’évolution des effectifs d’EDF SA sur une période donnée ?**

*Respect de la PEP-8*

In [33]:
%load_ext pycodestyle_magic
%pycodestyle_on

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [34]:
import pandas as pd

#### 1) Analyse de l'évolution des effectifs par statut et age ####

In [119]:
??


IPython -- An enhanced Interactive Python

IPython offers a fully compatible replacement for the standard Python
interpreter, with convenient shell features, special commands, command
history mechanism and output results caching.

At your system command line, type 'ipython -h' to see the command line
options available. This document only describes interactive features.

GETTING HELP
------------

Within IPython you have various way to access help:

  ?         -> Introduction and overview of IPython's features (this screen).
  object?   -> Details about 'object'.
  object??  -> More detailed, verbose information about 'object'.
  %quickref -> Quick reference of all IPython specific syntax and magics.
  help      -> Access Python's own help system.

If you are in terminal IPython you can quit this screen by pressing `q`.


MAIN FEATURES
-------------

* Access to the standard Python help with object docstrings and the Python
  manuals. Simply type 'help' (no quotes) to invoke it.

* Ma

In [12]:
effectifs = pd.read_csv("data/effectifs.csv", delimiter=";")
effectifs.columns

Index(['Année', 'Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicateur', 'Indicator', 'Type de contrat',
       'Type of contract', 'Collège', 'Employee category',
       'Sous-catégorie collège', 'Employee subcategory', 'Genre', 'Gender',
       'Plage M3E', 'M3E classification', 'Nationalité', 'Nationality',
       'Ancienneté', 'Seniority', 'Tranche d'âge', 'Age bracket', 'Valeur',
       'Unité', 'Unit', 'Chapitre du bilan social'],
      dtype='object')

#### 2) Corrélation entre formation et promotions ####

Ici, nous allons analyser l'effet des formations (nombre d'heures, nombre d'employés ayant été upgrade) avec une petite comparaison entre hommes et femmes

In [115]:
promotion = pd.read_csv("data/promotion.csv", delimiter=";")
promotion = promotion[promotion["Indicateur"]=="Promotions dans un collège supérieur"]
promotion = promotion[promotion["Type de contrat"]=="Statutaire"]
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Plage M3E',
       'M3E classification', 'Gender', 'Unit',
       'Chapitre du bilan social', 'Type de contrat', 'Indicateur']
promotion = promotion.drop(columns=drop)
promotion = promotion.groupby(["Année", "Collège", "Genre", "Unité"]).sum().reset_index()
promotion

,Année,Collège,Genre,Unité,Valeur
0,2017,Exécution vers Maîtrise,Femme,nombre,118.0
1,2017,Exécution vers Maîtrise,Homme,nombre,269.0
2,2017,Maîtrise vers Cadre,Femme,nombre,152.0
3,2017,Maîtrise vers Cadre,Homme,nombre,445.0
4,2018,Exécution vers Maîtrise,Femme,nombre,103.0
5,2018,Exécution vers Maîtrise,Homme,nombre,251.0
6,2018,Maîtrise vers Cadre,Femme,nombre,165.0
7,2018,Maîtrise vers Cadre,Homme,nombre,441.0
8,2019,Exécution vers Maîtrise,Femme,nombre,132.0
9,2019,Exécution vers Maîtrise,Homme,nombre,339.0


2:46: E225 missing whitespace around operator
2:80: E501 line too long (86 > 79 characters)
3:51: E225 missing whitespace around operator
5:8: E128 continuation line under-indented for visual indent
6:8: E128 continuation line under-indented for visual indent
7:8: E128 continuation line under-indented for visual indent
8:8: E128 continuation line under-indented for visual indent
10:80: E501 line too long (89 > 79 characters)


In [111]:
formation = pd.read_csv("data/formation.csv", delimiter = ";")
formation = formation[(formation["Collège"] == "Maitrise") | (formation["Collège"] == "Exécution")]
formation = formation[(formation["Indicateur"] == "Heures de formation") | (formation["Indicateur"] == "Salariés ayant suivi au moins un stage")]
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Employee category', 'Plage M3E', 'M3E classification',
       'Gender', 'Unit', 'Chapitre du bilan social']
formation = formation.drop(columns=drop)
formation

,Année,Indicateur,Collège,Genre,Valeur,Unité
2,2023,Salariés ayant suivi au moins un stage,Maitrise,Femme,5618.0,nombre
12,2023,Salariés ayant suivi au moins un stage,Exécution,Femme,1375.0,nombre
18,2023,Heures de formation,Maitrise,NaN,1911393.0,heures
31,2023,Salariés ayant suivi au moins un stage,Maitrise,Homme,18665.0,nombre
46,2023,Salariés ayant suivi au moins un stage,Exécution,Homme,3315.0,nombre
53,2023,Heures de formation,Exécution,NaN,442154.0,heures
89,2022,Salariés ayant suivi au moins un stage,Maitrise,Femme,5656.0,Nombre
90,2022,Salariés ayant suivi au moins un stage,Exécution,Homme,2838.0,Nombre
104,2022,Salariés ayant suivi au moins un stage,Maitrise,Homme,18289.0,Nombre
105,2022,Salariés ayant suivi au moins un stage,Exécution,Femme,1323.0,Nombre


1:56: E251 unexpected spaces around keyword / parameter equals
1:58: E251 unexpected spaces around keyword / parameter equals
2:80: E501 line too long (99 > 79 characters)
3:80: E501 line too long (145 > 79 characters)
5:8: E128 continuation line under-indented for visual indent
6:8: E128 continuation line under-indented for visual indent
7:8: E128 continuation line under-indented for visual indent


#### 3) Impact des conditions de travail sur l’absentéisme ####

lien entre paramètres d etravail (rémunération / vacances / autres ?) et arrets maladie par collège pour les salariés EDF


In [113]:
absence_maladie = pd.read_csv("data/absences.csv", delimiter=";")
absence_maladie = absence_maladie[absence_maladie["Type de contrat"]=="Statutaires"]
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Gender',
       'Unit', 'Type de contrat', 'Chapitre du bilan social', 'Genre']
absence_maladie = absence_maladie.drop(columns=drop)
absence_maladie = absence_maladie[absence_maladie["Indicateur"] == "Absence pour maladie"]
absence_maladie = absence_maladie.groupby(["Année", "Collège", "Indicateur", "Unité"]).sum().reset_index()
absence_maladie

,Année,Collège,Indicateur,Unité,Valeur
0,2017,Cadre,Absence pour maladie,heures,1081209.0
1,2017,Exécution,Absence pour maladie,heures,756905.0
2,2017,Maîtrise,Absence pour maladie,heures,2330284.0
3,2018,Cadre,Absence pour maladie,heures,1118943.0
4,2018,Exécution,Absence pour maladie,heures,702581.0
5,2018,Maîtrise,Absence pour maladie,heures,2288034.0
6,2019,Cadre,Absence pour maladie,heures,1110863.0
7,2019,Exécution,Absence pour maladie,heures,582413.0
8,2019,Maîtrise,Absence pour maladie,heures,2191619.0
9,2020,Cadre,Absence pour maladie,heures,1073816.0


2:69: E225 missing whitespace around operator
2:80: E501 line too long (84 > 79 characters)
4:8: E128 continuation line under-indented for visual indent
5:8: E128 continuation line under-indented for visual indent
6:8: E128 continuation line under-indented for visual indent
8:80: E501 line too long (90 > 79 characters)
9:80: E501 line too long (106 > 79 characters)


In [116]:
vacances = pd.read_csv("data/absences.csv", delimiter=";")
vacances = vacances[vacances["Type de contrat"]=="Statutaires"]
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Gender',
       'Unit', 'Type de contrat', 'Chapitre du bilan social', 'Genre']
vacances = vacances.drop(columns=drop)
vacances = vacances[vacances["Indicateur"] == "Absence pour congés autorisés"]
vacances = vacances.groupby(["Année", "Collège", "Indicateur", "Unité"]).sum().reset_index()
vacances

,Année,Collège,Indicateur,Unité,Valeur
0,2017,Cadre,Absence pour congés autorisés,heures,327130.0
1,2017,Exécution,Absence pour congés autorisés,heures,71491.0
2,2017,Maîtrise,Absence pour congés autorisés,heures,362252.0
3,2018,Cadre,Absence pour congés autorisés,heures,350507.0
4,2018,Exécution,Absence pour congés autorisés,heures,69055.0
5,2018,Maîtrise,Absence pour congés autorisés,heures,370494.0
6,2019,Cadre,Absence pour congés autorisés,heures,270791.0
7,2019,Exécution,Absence pour congés autorisés,heures,51543.0
8,2019,Maîtrise,Absence pour congés autorisés,heures,311058.0
9,2020,Cadre,Absence pour congés autorisés,heures,235808.0


2:48: E225 missing whitespace around operator
4:8: E128 continuation line under-indented for visual indent
5:8: E128 continuation line under-indented for visual indent
6:8: E128 continuation line under-indented for visual indent
9:80: E501 line too long (92 > 79 characters)


In [117]:
conditions = pd.read_csv("data/working_conditions.csv", delimiter = ";")
conditions = conditions[conditions["Type de contrat"]=="Statutaires"]
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator', 'Type de contrat',
       'Type of contract', 'Time range',
       'Employee category', 'Gender', 'Unit',
       'Chapitre du bilan social']
conditions = conditions.drop(columns=drop)
conditions

,Année,Indicateur,Tranche horaire,Collège,Genre,Valeur,Unité
2,2023,Salariés à temps partiel en décembre,Moins de 20 heures,Maitrise,Homme,32.0,Nombre
5,2023,Salariés à temps partiel en décembre,De 20 heures à moins de 30 heures,Cadre,Homme,54.0,Nombre
6,2023,Salariés à temps partiel en décembre,De 20 heures à moins de 30 heures,Maitrise,Femme,330.0,Nombre
13,2023,Salariés en réduction collective du temps de t...,NaN,Cadre,Femme,91.0,Nombre
14,2023,Salariés en réduction collective du temps de t...,NaN,Maitrise,Femme,632.0,Nombre
...,...,...,...,...,...,...,...
667,2017,Salariés en réduction collective du temps de t...,NaN,Maitrise,Homme,938.0,Nombre
676,2017,Salariés à temps partiel en décembre,De 20 heures à moins de 30 heures,Maitrise,Homme,77.0,Nombre
680,2017,Salariés à temps partiel en décembre,De 30 heures à moins de 35 heures,Maitrise,Homme,189.0,Nombre
682,2017,Salariés en réduction collective du temps de t...,NaN,Cadre,Homme,239.0,Nombre


1:66: E251 unexpected spaces around keyword / parameter equals
1:68: E251 unexpected spaces around keyword / parameter equals
2:54: E225 missing whitespace around operator


4:8: E128 continuation line under-indented for visual indent
5:8: E128 continuation line under-indented for visual indent
6:8: E128 continuation line under-indented for visual indent
7:8: E128 continuation line under-indented for visual indent


In [ ]:
salaire = pd.read_csv("data/promotion.csv", delimiter=";")
salaire = salaire[salaire["Indicateur"]=="Rémunération mensuelle moyenne brute"]
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Plage M3E',
       'M3E classification', 'Gender', 'Unit',
       'Chapitre du bilan social', 'Type de contrat', 'Genre']
salaire = salaire.drop(columns=drop)
salaire = salaire.groupby(["Année", "Collège", "Indicateur", "Unité"]).sum().reset_index()
salaire

,Année,Collège,Indicateur,Unité,Valeur
0,2017,Cadre,Rémunération mensuelle moyenne brute,€,41549.0
1,2017,Execution,Rémunération mensuelle moyenne brute,€,10345.0
2,2017,Maitrise,Rémunération mensuelle moyenne brute,€,17210.0
3,2018,Cadre,Rémunération mensuelle moyenne brute,€,41849.0
4,2018,Execution,Rémunération mensuelle moyenne brute,€,10351.0
5,2018,Maitrise,Rémunération mensuelle moyenne brute,€,17735.0
6,2019,Cadre,Rémunération mensuelle moyenne brute,€,40688.0
7,2019,Execution,Rémunération mensuelle moyenne brute,€,11302.0
8,2019,Maitrise,Rémunération mensuelle moyenne brute,€,17586.0
9,2020,Cadre,Rémunération mensuelle moyenne brute,€,39837.0


2:40: E225 missing whitespace around operator
2:80: E501 line too long (80 > 79 characters)
4:8: E128 continuation line under-indented for visual indent
5:8: E128 continuation line under-indented for visual indent
6:8: E128 continuation line under-indented for visual indent
7:8: E128 continuation line under-indented for visual indent
9:80: E501 line too long (90 > 79 characters)


In [76]:
conditions.columns

Index(['Année', 'Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicateur', 'Indicator', 'Type de contrat',
       'Type of contract', 'Tranche horaire', 'Time range', 'Collège',
       'Employee category', 'Genre', 'Gender', 'Valeur', 'Unité', 'Unit',
       'Chapitre du bilan social'],
      dtype='object')

#### 4) Focus sur les salariés non-statuaires ####

https://ufpi.reference-syndicale.fr/2017/06/stop-aux-cdi-non-statutaires-a-tous-les-etages/

L'idée ici sera de comparer plusieurs indicateurs (salaires, congés, arrets maladie) selon si ils sont ststuaires ou non, en fonction du collège

In [104]:
salaire = pd.read_csv("data/promotion.csv", delimiter=";")
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Plage M3E',
       'M3E classification', 'Gender', 'Unit',
       'Chapitre du bilan social', 'Type de contrat', 'Genre']
salaire = salaire.drop(columns=drop)
salaire = salaire.groupby(["Année", "Collège", "Indicateur", "Unité"]).sum().reset_index()
salaire

,Année,Indicateur,Collège,Valeur,Unité
0,2023,Stagiaires scolaires,Maîtrise,492.0,nombre
1,2023,Salariés détachés en mobilité à l'international,NaN,23.0,nombre
2,2023,Salariés détachés accueillis,NaN,1544.0,nombre
3,2023,Salariés détachés accueillis,NaN,44.0,nombre
4,2023,Stagiaires scolaires,Cadre,6.0,nombre
...,...,...,...,...,...
86,2017,Salariés détachés accueillis,NaN,801.0,nombre
87,2017,Salariés détachés accueillis,NaN,18.0,nombre
88,2017,Stagiaires scolaires,Cadre,3.0,nombre
89,2017,Stagiaires scolaires,Exécution,148.0,nombre


3:8: E128 continuation line under-indented for visual indent
3:61: E231 missing whitespace after ','
4:8: E128 continuation line under-indented for visual indent


In [118]:
absence_full = pd.read_csv("data/absences.csv", delimiter=";")
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Gender',
       'Unit', 'Type de contrat', 'Chapitre du bilan social', 'Genre']
absence_full = absence_full.drop(columns=drop)
absence_full = absence_full.groupby(["Année", "Collège", "Indicateur", "Unité"]).sum().reset_index()
absence_full

,Année,Collège,Indicateur,Unité,Valeur
0,2017,Cadre,Absence pour congé de maternité ou d'adoption,heures,345432.0
1,2017,Cadre,Absence pour congé de paternité et d'accueil d...,heures,59396.0
2,2017,Cadre,Absence pour congés autorisés,heures,328844.0
3,2017,Cadre,Absence pour maladie,heures,1095570.0
4,2017,Exécution,Absence pour congé de maternité ou d'adoption,heures,106915.0
...,...,...,...,...,...
79,2023,Exécution,Absence pour maladie,heures,395846.0
80,2023,Maîtrise,Absence pour congé de maternité ou d'adoption,heures,159668.0
81,2023,Maîtrise,Absence pour congé de paternité et d'accueil d...,heures,122928.0
82,2023,Maîtrise,Absence pour congés autorisés,heures,278074.0


3:8: E128 continuation line under-indented for visual indent
4:8: E128 continuation line under-indented for visual indent
5:8: E128 continuation line under-indented for visual indent
7:80: E501 line too long (100 > 79 characters)
